Authors: Luís Eduardo Anunciado Silva e Mayra Datas de Azevedo

In [0]:
!pip install unidecode

In [0]:
import pandas as pd
import numpy as np

from unidecode import unidecode

import requests

import folium

# Gathering the data

In [0]:
neighborhood_df = pd.read_csv('neighbourhoods.csv')

In [0]:
# load the GeoJSON data and use 'UTF-8'encoding
geojson_natal_file = requests.get('https://github.com/nymarya/data-science-one/blob/master/Lesson%2314/natal.geojson?raw=true')
geo_json_natal = geojson_natal_file.json()

In [0]:
# Insert additional information ('population') into GeoJSON file
for pop in geo_json_natal['features']:
    codarea = pop['properties']['name']
    codarea = codarea if codarea != 'Felipe Camarão' else 'Filipe Camarão'
    neigh = neighborhood_df.query('name_norm == "' + unidecode(codarea) + '"')
    pop['properties']['population'] = str(neigh.inhabitants.values[0])
    pop['properties']['name'] = str(neigh.name.values[0]) # assure the name is the same

In [0]:
def build_map(df_key, geojson_key, legend):
  # Create a map object
  m = folium.Map(
      location=[-5.826592, -35.212558],
      zoom_start=11.3,
      tiles='Stamen Terrain'
  )

  # create a threshold of legend
  bins = np.linspace(neighborhood_df[df_key].min(),
                     neighborhood_df[df_key].max(),
                     6).tolist()


  choropleth = folium.Choropleth(
      geo_data=geo_json_natal,
      data=neighborhood_df,
      name= "neighborhoods",
      columns=['name', df_key],
      key_on='feature.properties.name',
      fill_color='BuPu',
      fill_opacity=0.7,
      line_opacity=0.5,
      legend_name='#'+legend+' in Natal-RN',
      bins=bins
  ).add_to(m)


  folium.LayerControl().add_to(m)

  choropleth.geojson.add_child(
      folium.features.GeoJsonTooltip(fields=['name',geojson_key], 
                                              aliases=['Name:',legend.capitalize() + ":"], 
                                              localize=True)
  )

  return m

## Getting an ideia about the population

### Population

In [9]:
build_map('inhabitants', 'population', 'population')

In [0]:
# Insert additional information ('population') into GeoJSON file
for pop in geo_json_natal['features']:
    codarea = pop['properties']['name']
    codarea = codarea if codarea != 'Felipe Camarão' else 'Filipe Camarão'
    neigh = neighborhood_df.query('name_norm == "' + unidecode(codarea) + '"')
    pop['properties']['population'] = str(neigh.inhabitants.values[0])
    pop['properties']['income'] = str(neigh.income.values[0])

### Income


In [11]:
build_map('income', 'income', 'income')

## Measuring pedestrian accessibility

In [0]:
# Insert additional information ('population') into GeoJSON file
for pop in geo_json_natal['features']:
    codarea = pop['properties']['name']
    codarea = codarea if codarea != 'Felipe Camarão' else 'Filipe Camarão'
    neigh = neighborhood_df.query('name_norm == "' + unidecode(codarea) + '"')
    pop['properties']['dist_school'] = str(neigh.distance_school.values[0])
    pop['properties']['dist_hospital'] = str(neigh.distance_hospital.values[0])
    pop['properties']['dist_police'] = str(neigh.distance_police.values[0])
    pop['properties']['dist_restaurant'] = str(neigh.distance_restaurant.values[0])
    pop['properties']['name'] = str(neigh.name.values[0]) # assure the name is the same
    

### Schools

In [14]:
build_map('distance_school', 'dist_school', 'distance from center to nearest school')

A person who lives at the center of Lagoa Azul has to walk about 8KM to reach a school, while in Lagoa Nova would be 200 meters or so.

### Hospitals

In [15]:
build_map('distance_hospital', 'dist_hospital', 'distance from center to nearest hospital')

The same pattern is found when analysing the distance to a hospital. A person who lives in the center of Ponta Negra, Lagoa Azul or Nossa Senhora da Apresentação has to walk 8km to reach a hospital, even though it would be necessary only 100m from the center of Cidade da Esperança.

The neighborhood Neópolis draws some attention since is a relatively small area and it takes 5km to go from the center to the nearest hospital.

### Police stations

In [16]:
build_map('distance_police', 'dist_police', 'distance from center to nearest police station')

Again, if we start at the center of Lagoa Azul, we would have to walk about 8km to get to the nearest police station, while in any neighborhood located in the center of Natal, this distance would be less than 2km.

### Restaurants

In [17]:
build_map('distance_restaurant', 'dist_restaurant', 'distance from center to nearest restaurant')